In [1]:
pip install hazm

     |████████████████████████████████| 317kB 5.9MB/s 
     |████████████████████████████████| 1.4MB 11.7MB/s 
     |████████████████████████████████| 235kB 135kB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394472 sha256=20be8c8648b998476f3fc601423e20bc21461c442464e4e4ca1e4b8978eace28
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154748 sha256=e2a663693e840b64a335ce91409a72a935bd538c7383f3d7141a4cfeeff12580
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [13]:
pip install persiantools

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from __future__ import unicode_literals
import torch
import nltk
import pandas as pd
import hazm
from hazm import *
from persiantools import characters
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
import time
from datetime import timedelta
from datetime import datetime
def preprocess_on_each_sent(sentence, _lemmatizer):
    new_sentence = ""
    punctuations = ['.', '؟', '/', '،', '!']
    stop_words_list = hazm.utils.stopwords_list()
    list_of_words = word_tokenize(sentence)
    for i in list_of_words:
        if i not in stop_words_list and i not in punctuations:
            new_word = _lemmatizer.lemmatize(i)
            new_sentence = new_sentence + " " + new_word
    return new_sentence


def read_input_file():
    data_file = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Final_Exam/mobile_phone_dataset.csv')
    return data_file

def process_data_file(input_data_file):
    days = []
    time = []
    lemmatizer = Lemmatizer()
    stemmer = Stemmer()
    normalizer = Normalizer()
    stemmer = Stemmer()
    input_data_file = input_data_file[input_data_file.price >= 0].reset_index(drop=True)
    input_data_file['desc'] = input_data_file.apply(lambda each_row: normalizer.normalize(each_row['desc']), axis=1)
    input_data_file['title'] = input_data_file.apply(lambda each_row: normalizer.normalize(each_row['title']), axis=1)
    input_data_file['desc'] = input_data_file.apply(lambda each_row: preprocess_on_each_sent(each_row['desc'], lemmatizer), axis=1)
    input_data_file['title'] = input_data_file.apply(lambda each_row: preprocess_on_each_sent(each_row['title'], lemmatizer), axis=1)



    input_data_file['created_at'] = df.apply(lambda each_row: nltk.word_tokenize(each_row['created_at']), axis=1)
    input_data_file.apply(lambda each_row: days.append(each_row['created_at'][0]), axis=1)
    input_data_file.apply(lambda each_row: time.append(each_row['created_at'][1]), axis=1)
    input_data_file['created_at_day'] = days
    input_data_file['created_at_time'] = time
    input_data_file = input_data_file.drop('created_at', axis=1)

    input_data_file['brand'] = df.apply(lambda each_row: each_row['brand'].split('::')[0], axis=1)

    return input_data_file


def split_train_test_data(data_file):
    features = data_file.drop('price', axis=1)
    target = data_file['price']
    features_train, features_test, target_train, target_test = train_test_split(features, target, train_size=0.8)
    return features_train, features_test, target_train, target_test


def manage_data(features_train, features_test, data_file):
    vectorizer = CountVectorizer(lowercase=False, binary=True)
    #df_brand_oneHot = vectorizer.fit_transform(data_file['brand'].values)
    train_brand_oneHot = vectorizer.fit_transform(features_train['brand'].values)
    test_brand_oneHot = vectorizer.transform(features_test['brand'].values)

    #df_city_oneHot = vectorizer.fit_transform(data_file['city'].values)
    train_city_oneHot = vectorizer.fit_transform(features_train['city'].values)
    test_city_oneHot = vectorizer.transform(features_test['city'].values)

    #df_day_oneHot = vectorizer.fit_transform(data_file['created_at_day'].values)
    train_day_oneHot = vectorizer.fit_transform(features_train['created_at_day'].values)
    test_day_oneHot = vectorizer.transform(features_test['created_at_day'].values)

    #df_time_oneHot = vectorizer.fit_transform(data_file['created_at_time'].values)
    train_time_oneHot = vectorizer.fit_transform(features_train['created_at_time'].values)
    test_time_oneHot = vectorizer.transform(features_test['created_at_time'].values)

    df_image_count = [str(i) for i in data_file['image_count'].values]
    train_image_count = [str(i) for i in features_train['image_count'].values]
    test_image_count = [str(i) for i in features_test['image_count'].values]

    print(train_image_count)
    df_image_count_oneHot = vectorizer.fit_transform(df_image_count)
    train_image_count_oneHot = vectorizer.transform(train_image_count)
    test_image_count_oneHot = vectorizer.transform(test_image_count)

    vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=3, max_features=1000)
    #df_title_tfidf = vectorizer.fit_transform(data_file['title'].values)
    train_title_tfidf = vectorizer.fit_transform(features_train['title'].values)
    test_title_tfidf = vectorizer.transform(features_test['title'].values)

    vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_features=2000)
    #df_desc_tfidf = vectorizer.fit_transform(data_file['desc'].values)
    train_desc_tfidf = vectorizer.fit_transform(features_train['desc'].values)
    test_desc_tfidf = vectorizer.transform(features_test['desc'].values)

    train_sparse = hstack((
        train_brand_oneHot, train_city_oneHot, train_day_oneHot, train_time_oneHot, train_image_count_oneHot,
        train_title_tfidf,
        train_desc_tfidf)).tocsr()
    test_sparse = hstack((test_brand_oneHot, test_city_oneHot, test_day_oneHot, test_time_oneHot,
                          test_image_count_oneHot, test_title_tfidf,
                          test_desc_tfidf)).tocsr()

    return train_sparse, test_sparse


def model_prediction(pro_features_train, pro_features_test, target_train, target_test):
    lgr = LogisticRegression(random_state=0, max_iter=10000)
    lgr.fit(pro_features_train, target_train)

    actual_train_data = list(target_train)
    predict_train_data = lgr.predict(pro_features_train)
    actual_test_data = list(target_test)
    predict_test_data = lgr.predict(pro_features_test)

        acc_test = lgr.score(pro_features_test, actual_test_data)
    acc_train = lgr.score(pro_features_train, actual_train_data)
    print('Accuracy Test= %.3f' % (acc_test * 100))
    print('Accuracy Train= %.3f' % (acc_train * 100))

def model2_prediction(pro_features_train, pro_features_test, target_train, target_test):
    trg = DecisionTreeRegressor(random_state=0, max_depth = 10)
    trg.fit(pro_features_train, target_train)

    actual_train_data = list(target_train)
    predict_train_data = trg.predict(pro_features_train)
    actual_test_data = list(target_test)
    predict_test_data = trg.predict(pro_features_test)

    acc_test = trg.score(pro_features_test, actual_test_data)
    acc_train = trg.score(pro_features_train, actual_train_data)
    print('Accuracy Test= %.3f' % (acc_test * 100))
    print('Accuracy Train= %.3f' % (acc_train * 100))




In [20]:
df = read_input_file()
df_slice = df.head(20000)

# New section

In [ ]:
input_data = process_data_file(df)
print(input_data)

In [22]:
main_features_train, main_features_test, main_target_train, main_target_test = split_train_test_data(input_data)

In [ ]:
processed_features_train, processed_features_test = manage_data(main_features_train, main_features_test, input_data)

In [ ]:
model_prediction(processed_features_train, processed_features_test, main_target_train, main_target_test)

In [28]:
model2_prediction(processed_features_train, processed_features_test, main_target_train, main_target_test)

NameError: ignored